In [2]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [3]:
week = 23 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

29 March 2023 ; 12:21:14


# Features, model

In [4]:
features = ['VORP','WS','OWS','BPM','PER','PTS','FT','FTA','OBPM','FG','USG%','Seed','MP','GS']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [5]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/mvp/mvp_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') 

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,32,0,9.4,1.5,3.4,0.440,...,4.36,115.92,111.72,4.2,4.14,115.78,111.79,4.0,1,1
1,Bobby Portis,PF,27,MIL,64,20,26.0,5.6,11.3,0.496,...,4.36,115.92,111.72,4.2,4.14,115.78,111.79,4.0,0,1
2,Brook Lopez,C,34,MIL,73,73,30.5,6.0,11.5,0.524,...,4.36,115.92,111.72,4.2,4.14,115.78,111.79,4.0,0,0
3,Giannis Antetokounmpo,PF,28,MIL,59,59,32.2,11.2,20.3,0.550,...,4.36,115.92,111.72,4.2,4.14,115.78,111.79,4.0,0,0
4,Goran Dragić,PG,36,MIL,52,0,15.3,2.4,5.8,0.422,...,4.36,115.92,111.72,4.2,4.14,115.78,111.79,4.0,0,1


In [6]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] >= season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [7]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'Share']]
res.head()

,Player,Tm,PER,VORP,WS,PTS,Seed,Share
0,Nikola Jokić,DEN,31.9,10.770149,18.480597,24.9,1,0.853785
1,Giannis Antetokounmpo,MIL,28.8,6.949153,11.396610,31.1,1,0.672581
2,Joel Embiid,PHI,31.6,8.065574,15.593443,33.3,3,0.628699
3,Luka Dončić,DAL,29.3,8.468852,13.173770,32.9,10,0.400589
4,Jayson Tatum,BOS,23.3,5.388571,11.128571,30.1,2,0.277594


In [8]:
res.to_csv(path + f'Results/mvp/results_week_{week}.csv', index = None)